# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [20]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
import pickle
nltk.download('averaged_perceptron_tagger')
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator,TransformerMixin

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_reponse_pipeline.db')
df = pd.read_sql_table('disaster_response_data', engine)  
X = df.message 
Y = df.drop(columns=["id", "message","original", "genre"])
print(X.tail())

26210    The training demonstrated how to enhance micro...
26211    A suitable candidate has been selected and OCH...
26212    Proshika, operating in Cox's Bazar municipalit...
26213    Some 2,000 women protesting against the conduc...
26214    A radical shift in thinking came about as a re...
Name: message, dtype: object


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # remove url from text
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # remove numbers from the text (also incl. expression that start with a number)
    number_regex = '^[0-9]*$'
    detected_numbers = re.findall(number_regex, text)
    for number in detected_numbers:
        text = text.replace(number, "number")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
        ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
# predict on test data
y_pred = pipeline.predict(X_test)

# display results
for i,col in enumerate(Y.columns):
    print("Category:", col)
    print(classification_report(y_test.iloc[:, i].values, y_pred[:, i]))


Category: related
             precision    recall  f1-score   support

          0       0.62      0.34      0.44      1554
          1       0.82      0.94      0.87      5000

avg / total       0.77      0.79      0.77      6554

Category: request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5432
          1       0.82      0.39      0.53      1122

avg / total       0.87      0.88      0.86      6554

Category: offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

Category: aid_related
             precision    recall  f1-score   support

          0       0.71      0.88      0.79      3809
          1       0.75      0.51      0.60      2745

avg / total       0.73      0.72      0.71      6554

Category: medical_help
             precision    recall  f1-score 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
parameters = {
        'clf__estimator__n_estimators': [50, 200],
        'clf__estimator__min_samples_split': [2, 4]}

cv = GridSearchCV(pipeline, param_grid=parameters, scoring="f1_micro")

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [8]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [15]:
pipeline_adj = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            ('starting_verb', StartingVerbExtractor())
        ])),
        ('clf', MultiOutputClassifier(LogisticRegression()))
    ])



In [24]:
pipeline_adj.fit(X_train, y_train)

# predict on test data
y_pred = pipeline_adj.predict(X_test)

# display results
for i,col in enumerate(Y.columns):
    print("Category:", col)
    print(classification_report(y_test.iloc[:, i].values, y_pred[:, i]))

Category: related
             precision    recall  f1-score   support

          0       0.75      0.39      0.52      1554
          1       0.84      0.96      0.89      5000

avg / total       0.81      0.82      0.80      6554

Category: request
             precision    recall  f1-score   support

          0       0.92      0.98      0.94      5432
          1       0.83      0.57      0.67      1122

avg / total       0.90      0.91      0.90      6554

Category: offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6521
          1       0.00      0.00      0.00        33

avg / total       0.99      0.99      0.99      6554

Category: aid_related
             precision    recall  f1-score   support

          0       0.79      0.86      0.82      3809
          1       0.77      0.68      0.72      2745

avg / total       0.78      0.78      0.78      6554

Category: medical_help
             precision    recall  f1-score 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [26]:
pickle.dump(pipeline, open('model.pkl', 'wb'))


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.